In [1]:
"""
Guidelines retrieval utility for insurance underwriting.
This module provides functions to retrieve relevant guidelines from the vector store
and interpret them for the specific assessment needs.
"""

import logging
from typing import Dict, List, Any, Optional

logger = logging.getLogger("InsuranceAgentSystem")

class GuidelinesRetriever:
    """
    Provides context-aware retrieval of underwriting guidelines
    from the vector database based on specific query needs.
    """
    
    def __init__(self, retriever):
        """
        Initialize with a configured retriever from the vector store.
        
        Args:
            retriever: A configured retriever from llama_index
        """
        self.retriever = retriever
        
    async def get_hazard_guidelines(self, building_type: str, construction: str) -> str:
        """
        Retrieve hazard classification guidelines relevant to the specific building and construction.
        
        Args:
            building_type: The type of building (e.g., "Office", "Manufacturing")
            construction: The construction type (e.g., "Steel Frame", "Concrete")
            
        Returns:
            A formatted string containing relevant guidelines
        """
        try:
            query = f"hazard classification guidelines for {building_type} buildings with {construction} construction"
            logger.info(f"Querying hazard guidelines with: {query}")
            
            # Query the vector store
            retrieval_results = self.retriever.retrieve(query)
            
            if not retrieval_results:
                return "No specific hazard guidelines found. Using default assessment criteria."
            
            # Extract and format the results
            guidelines_info = "\n\n".join([node.text for node in retrieval_results])
            
            logger.info("Successfully retrieved hazard guidelines")
            return guidelines_info
        except Exception as e:
            logger.error(f"Error querying hazard guidelines: {str(e)}")
            return "Error retrieving hazard guidelines. Using default assessment criteria."
    
    async def get_vulnerability_guidelines(self, building_type: str, building_age: int) -> str:
        """
        Retrieve vulnerability assessment guidelines relevant to the building type and age.
        
        Args:
            building_type: The type of building (e.g., "Office", "Manufacturing")
            building_age: The age of the building in years
            
        Returns:
            A formatted string containing relevant guidelines
        """
        try:
            query = f"vulnerability assessment criteria for {building_type} buildings {building_age} years old"
            logger.info(f"Querying vulnerability guidelines with: {query}")
            
            # Query the vector store
            retrieval_results = self.retriever.retrieve(query)
            
            if not retrieval_results:
                return "No specific vulnerability guidelines found. Using default assessment criteria."
            
            # Extract and format the results
            guidelines_info = "\n\n".join([node.text for node in retrieval_results])
            
            logger.info("Successfully retrieved vulnerability guidelines")
            return guidelines_info
        except Exception as e:
            logger.error(f"Error querying vulnerability guidelines: {str(e)}")
            return "Error retrieving vulnerability guidelines. Using default assessment criteria."
    
    async def get_cat_modeling_guidelines(self, location: str) -> str:
        """
        Retrieve catastrophe modeling guidelines relevant to the specific location.
        
        Args:
            location: The address or geographic location of the property
            
        Returns:
            A formatted string containing relevant guidelines
        """
        try:
            query = f"catastrophe risk assessment criteria for properties in {location}"
            logger.info(f"Querying CAT modeling guidelines with: {query}")
            
            # Query the vector store
            retrieval_results = self.retriever.retrieve(query)
            
            if not retrieval_results:
                return "No specific CAT modeling guidelines found. Using default assessment criteria."
            
            # Extract and format the results
            guidelines_info = "\n\n".join([node.text for node in retrieval_results])
            
            logger.info("Successfully retrieved CAT modeling guidelines")
            return guidelines_info
        except Exception as e:
            logger.error(f"Error querying CAT modeling guidelines: {str(e)}")
            return "Error retrieving CAT modeling guidelines. Using default assessment criteria."
    
    async def get_decision_guidelines(self, submission_data: Dict[str, Any]) -> str:
        """
        Enhanced function to retrieve decision-specific underwriting guidelines.
        Uses multiple targeted queries based on submission details.
        
        Args:
            submission_data: Dictionary containing submission details
            
        Returns:
            Relevant guidelines information
        """
        logger.info("Querying decision-specific guidelines")
        
        # Extract key properties for targeted queries
        property_details = submission_data.get("property_details", {})
        building_type = property_details.get("building_type", "")
        construction = property_details.get("construction", "")
        year_built = property_details.get("year_built", 0)
        building_age = datetime.now().year - year_built if year_built else 0
        occupancy = property_details.get("occupancy", "")
        
        # Prepare multiple targeted queries
        queries = [
            f"underwriting decision criteria for {building_type} buildings",
            f"eligibility requirements for {building_type} with {construction} construction",
            f"decision requirements for buildings {building_age} years old"
        ]
        
        # If we have specific occupancy information, add that query
        if occupancy:
            queries.append(f"occupancy guidelines for {occupancy}")
        
        # Add general decision criteria query
        queries.append("general underwriting criteria for insurance decisions")
        
        # Retrieve guidelines for each query and combine results
        all_guidelines = []
        
        for query in queries:
            logger.info(f"Querying guidelines with: {query}")
            
            # Use the existing vector store to retrieve guidelines
            retrieval_results = self.retriever.retrieve(query)
            
            if retrieval_results:
                guidelines_info = "\n\n".join([node.text for node in retrieval_results])
                if guidelines_info not in all_guidelines:
                    all_guidelines.append(guidelines_info)
        
        # Combine all unique guidelines
        combined_guidelines = "\n\n---\n\n".join(all_guidelines)
        
        if not combined_guidelines:
            return "No specific decision guidelines found. Using default assessment criteria."
            
        logger.info("Successfully retrieved decision guidelines")
        return combined_guidelines
    
    async def get_eligibility_criteria(self, building_type: str) -> Dict[str, Any]:
        """
        Retrieve eligibility criteria for the specific building type.
        
        Args:
            building_type: The type of building (e.g., "Office", "Manufacturing")
            
        Returns:
            A dictionary containing parsed eligibility criteria
        """
        try:
            query = f"eligibility criteria for {building_type} buildings"
            logger.info(f"Querying eligibility criteria with: {query}")
            
            # Query the vector store
            retrieval_results = self.retriever.retrieve(query)
            
            if not retrieval_results:
                return {"status": "Not found", "details": "Using default eligibility criteria"}
            
            # Extract the results
            criteria_text = "\n\n".join([node.text for node in retrieval_results])
            
            # Parse into structured format - in a real system this would be more sophisticated
            # Currently returning the raw text with a basic structure
            criteria = {
                "status": "Found",
                "text": criteria_text,
                "summary": {
                    "max_age": 35,
                    "max_stories": 3 if "sprinklered" in criteria_text.lower() else 2,
                    "protection_classes": "1-6"
                }
            }
            
            logger.info("Successfully retrieved eligibility criteria")
            return criteria
        except Exception as e:
            logger.error(f"Error querying eligibility criteria: {str(e)}")
            return {"status": "Error", "details": str(e)}
        
# Print confirmation that this notebook was loaded
print("Guidelines retriever loaded successfully.")

Guidelines retriever loaded successfully.
